In [1]:
import psycopg2, sqlite3
from psycopg2.extensions import AsIs
from uuid import uuid4
import pandas as pd

In [2]:
def createTeam1SamplingFeatures():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        # Create cluster geometry table
        # samplingfeatureuuid uuid DEFAULT uuid_generate_v4 (),
        cur.execute("""CREATE TABLE team1_sampling_features (
                samplingfeatureid SERIAL PRIMARY KEY,
                samplingfeatureuuid uuid NOT NULL,
                samplingfeaturecode VARCHAR (50) NOT NULL,
                samplingfeaturename VARCHAR (255) NOT NULL,
                samplingfeaturedescription VARCHAR (500) NOT NULL,
                elevation_m double precision,
                samplingfeaturegeotypecv VARCHAR (255),
                samplingfeaturetypecv VARCHAR (255) NOT NULL,
                sitetypecv VARCHAR (255) NOT NULL,
                samplingfeatureexternalidentifier VARCHAR (255) NOT NULL,
                propertyvalue VARCHAR (255),
                longitude NUMERIC NOT NULL,
                latitude NUMERIC NOT NULL
            )""")
        # Create geometry column
        cur.execute("""ALTER TABLE team1_sampling_features ADD COLUMN featuregeometry geometry(POINT,4326)""")
        # Index geometry using SPGIST.
        cur.execute("""CREATE INDEX index_featuregeometry 
                       ON team1_sampling_features 
                       USING SPGIST ( featuregeometry )""")

        conn.commit()
        cur.execute("""ANALYZE team1_sampling_features""")

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [3]:
createTeam1SamplingFeatures()

In [4]:
def selectRoundOneFeatures():
    conn = None
    try:
        conn = sqlite3.connect('../RoundOneDB/RoundOneDB_final.db')
        cur = conn.cursor()
        cur.execute("""SELECT DISTINCT q.station_id, q.station_name, g.src_name, g.src_locati, 
                       g.lat,g.lon, g.src_type, q.epa_srcid, g.depth
                       FROM quant_summary q 
                       INNER JOIN geolocation g ON g.EPA_srcID = q.EPA_srcID
                       ORDER BY q.epa_srcid""")

        rows = cur.fetchall()
        cur.close()
        conn.close()
    
        df = pd.DataFrame(columns = ['station_id','station_name', 'src_name', 'src_locati',
                                     'lon','lat','src_type', 'epa_srcid', 'depth']) 
        for row in rows:
            df = df.append({'station_id': row[0],'station_name': row[1], 'src_name': row[2], 'src_locati': row[3],
                            'lon': row[4],'lat': row[5],'src_type': row[6], 'epa_srcid': row[7], 'depth': row[8]}, 
                            ignore_index = True)

        return df
    except Error as e:
        print(e)

In [5]:
df = selectRoundOneFeatures()
df.head()

,station_id,station_name,src_name,src_locati,lon,lat,src_type,epa_srcid,depth
0,NC0103010,"SPARTA, TOWN OF",WELL #9,MITCHELL MOUNTAIN RD,-81.134111,36.512317,G,11606,329
1,NC0105010,"WEST JEFFERSON, TOWN OF",WELL #3A-WEST SEVENTH ST,WEST 7TH ST OUTSIDE OF FENCE,-81.496376,36.408568,G,11736,500
2,NC0105015-2,"JEFFERSON, TOWN OF - 2",WELL #8 WOOD CROFT,HILLSIDE LN OFF HWY 221,-81.457671,36.412727,G,11747,255
3,NC0105020,"LANSING, TOWN OF",WELL #5,ACROSS RIVER FROM TOWN HALL,-81.510377,36.497001,G,11748,605
4,NC0106015,"BANNER ELK, TOWN OF",WELL #3,BELOW TOWN HALL,-81.868213,36.162782,G,11886,230


In [6]:
def insertTeam1Features(df):
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        
        for index, row in df.iterrows():
            samplingfeatureuuid = str(uuid4())
            samplingfeaturecode = row[0] #station_id=samplingfeaturecode
            samplingfeaturename = row[1] #station_name=samplingfeaturename
            if row[3] != None:
                samplingfeaturedescription = row[2]+'; '+row[3] #src_name+src_locati=samplingfeaturedescription
            else:
                samplingfeaturedescription = row[2]   
            longitude = row[4]
            latitude = row[5]
            if row[6].lower() == 's':
                sitetypecv = 'Stream' #src_type=sitetypecv s=Stream
            elif row[6].lower() == 'g':
                sitetypecv = 'Well' #src_type=sitetypecv g=Well
            else:
                sitetypecv = row[6]+' CHECK'
            samplingfeatureexternalidentifier = row[7] #epa_srcid=samplingfeatureexternalidentifier,
            propertyvalue = row[8] #depth=propertyvalue
                
            cur.execute("""INSERT INTO team1_sampling_features(samplingfeatureuuid, samplingfeaturecode, 
                                        samplingfeaturename, samplingfeaturedescription, samplingfeaturetypecv, 
                                        samplingfeaturegeotypecv, longitude, latitude, sitetypecv, 
                                        samplingfeatureexternalidentifier, propertyvalue)
                            VALUES (%(vsamplingfeatureuuid)s,%(vsamplingfeaturecode)s,%(vsamplingfeaturename)s, 
                                    %(vsamplingfeaturedescription)s,'Water quality station','Point',
                                    %(vlongitude)s,%(vlatitude)s,%(vsitetypecv)s,
                                    %(vsamplingfeatureexternalidentifier)s,%(vpropertyvalue)s)""",
                        {'vsamplingfeatureuuid': samplingfeatureuuid,
                         'vsamplingfeaturecode': samplingfeaturecode,
                         'vsamplingfeaturename': samplingfeaturename,
                         'vsamplingfeaturedescription': samplingfeaturedescription,
                         'vlongitude': AsIs(longitude),'vlatitude': AsIs(latitude),'vsitetypecv': sitetypecv,
                         'vsamplingfeatureexternalidentifier': AsIs(samplingfeatureexternalidentifier),
                         'vpropertyvalue': AsIs(propertyvalue)})
            # Create geometry data from lon, lat variables, and update table with those values.
            cur.execute("""UPDATE team1_sampling_features 
                           SET featuregeometry = ST_SetSRID(ST_MakePoint(longitude,latitude),4326)""")
            
            conn.commit()
            cur.execute("""ANALYZE team1_sampling_features""")
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()  

In [7]:
insertTeam1Features(df)

In [10]:
def insertFeaturesSites():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        cur.execute("""SELECT * FROM team1_sampling_features""")
        rows = cur.fetchall()
        for row in rows:
            samplingfeatureidin = row[0]
            samplingfeatureuuid = row[1]
            samplingfeaturecode = row[2]
            samplingfeaturename = row[3]
            samplingfeaturedescription = row[4]
            elevation_m = row[5]
            samplingfeaturegeotypecv = row[6]
            samplingfeaturetypecv = 'Water quality station' #row[7]
            sitetypecv = row[8]
            samplingfeatureexternalidentifier = row[9]
            propertyvalue = row[10]
            longitude = row[11]
            latitude = row[12]
            featuregeometry = row[13]
            
            cur.execute("""INSERT INTO sampling_features (samplingfeatureuuid, samplingfeaturecode, 
                                       samplingfeaturename, samplingfeaturedescription, featuregeometry,
                                       samplingfeaturegeotypecv, samplingfeaturetypecv)
                            VALUES (%(vamplingfeatureuuid)s, %(vsamplingfeaturecode)s, 
                                    %(vsamplingfeaturename)s, %(vsamplingfeaturedescription)s, 
                                    %(vfeaturegeometry)s, %(vsamplingfeaturegeotypecv)s, 
                                    %(vsamplingfeaturetypecv)s)
                            RETURNING samplingfeatureid""",
                        {'vamplingfeatureuuid':samplingfeatureuuid, 
                         'vsamplingfeaturecode':samplingfeaturecode, 
                         'vsamplingfeaturename':samplingfeaturename, 
                         'vsamplingfeaturedescription':samplingfeaturedescription, 
                         'vfeaturegeometry':featuregeometry,
                         'vsamplingfeaturegeotypecv':samplingfeaturegeotypecv, 
                         'vsamplingfeaturetypecv':samplingfeaturetypecv})
            samplingfeatureid = cur.fetchone()[0]
            
            cur.execute("""INSERT INTO sites (samplingfeatureid, longitude, latitude, 
                                              sitetypecv,spatialreferenceid)
                           VALUES (%(vsamplingfeatureid)s, %(vlongitude)s, %(vlatitude)s, 
                                   %(vsitetypecv)s,10001)""",
                        {'vsamplingfeatureid':samplingfeatureid,'vlongitude':longitude,'vlatitude':latitude,
                         'vsitetypecv':sitetypecv}),

            cur.execute("""INSERT INTO sampling_feature_external_identifiers 
                                          (samplingfeatureexternalidentifier,
                                           samplingfeatureexternalidentifieruri,
                                           externalidentifiersystemid,
                                           samplingfeatureid)
                           VALUES (%(vsamplingfeatureexternalidentifier)s, 'EPA Source ID', 10002,
                                   %(vsamplingfeatureid)s)""",
                        {'vsamplingfeatureexternalidentifier':samplingfeatureexternalidentifier,
                         'vsamplingfeatureid':samplingfeatureid})

            if propertyvalue != None:
                cur.execute("""INSERT INTO sampling_feature_extension_property_values 
                                              (propertyvalue,propertyid,samplingfeatureid)
                               VALUES (%(vpropertyvalue)s, 10015, %(vsamplingfeatureid)s)""",
                            {'vpropertyvalue':propertyvalue,
                             'vsamplingfeatureexternalidentifier':samplingfeatureexternalidentifier,
                             'vsamplingfeatureid':samplingfeatureid})
            else:
                continue
            
        conn.commit()
        cur.execute("""ANALYZE sampling_features""")
        cur.execute("""ANALYZE sites""")
        cur.execute("""ANALYZE sampling_feature_external_identifiers""")
        cur.execute("""ANALYZE sampling_feature_extension_property_values""")
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()  

In [11]:
insertFeaturesSites()

In [12]:
def insertFeaturesSpecimens():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        cur.execute("""SELECT * FROM sampling_features WHERE samplingfeaturecode like 'NC%'""")
        rows = cur.fetchall()
        for row in rows:
            samplingfeatureidin = row[0]
            samplingfeatureuuidin = row[1]
            samplingfeatureuuid = str(uuid4())
            samplingfeaturecode = row[2]+'_Specimen'
            samplingfeaturename = row[3]
            samplingfeaturedescription = row[4]
            elevation_m = row[5]
            samplingfeaturetypecv = 'Water quality station' #row[7]
            
            cur.execute("""INSERT INTO sampling_features (samplingfeatureuuid, samplingfeaturecode, 
                                       samplingfeaturename, samplingfeaturedescription,
                                       samplingfeaturetypecv)
                            VALUES (%(vamplingfeatureuuid)s, %(vsamplingfeaturecode)s, 
                                    %(vsamplingfeaturename)s, %(vsamplingfeaturedescription)s,  
                                    %(vsamplingfeaturetypecv)s)
                            RETURNING samplingfeatureid""",
                        {'vamplingfeatureuuid':samplingfeatureuuid, 
                         'vsamplingfeaturecode':samplingfeaturecode, 
                         'vsamplingfeaturename':samplingfeaturename, 
                         'vsamplingfeaturedescription':samplingfeaturedescription,  
                         'vsamplingfeaturetypecv':samplingfeaturetypecv})
            samplingfeatureid = cur.fetchone()[0]

            cur.execute("""INSERT INTO related_features (from_samplingfeature_id, to_samplingfeature_id,
                                                  relationshiptypecv)
                           VALUES (%(vsamplingfeatureid)s, %(vsamplingfeatureidin)s, 'Was collected at')""",
                        {'vsamplingfeatureid':samplingfeatureid, 'vsamplingfeatureidin':samplingfeatureidin})
            
            cur.execute("""INSERT INTO specimens (samplingfeatureid, isfieldspecimen,
                                                  specimenmediumcv, specimentypecv)
                           VALUES (%(vsamplingfeatureid)s,'t','Liquid aqueous', 'Individual sample')""",
                        {'vsamplingfeatureid':samplingfeatureid})
            
        conn.commit()
        cur.execute("""ANALYZE sampling_features""")
        cur.execute("""ANALYZE sampling_feature_external_identifiers""")
        cur.execute("""ANALYZE sampling_feature_extension_property_values""")
        cur.execute("""ANALYZE related_features""")
        cur.execute("""ANALYZE specimens""")
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()  

In [13]:
insertFeaturesSpecimens()